In [1]:
#import

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D
#from sklearn.cross_validation import StratifiedKFold, cross_val_score
#from sklearn import grid_search
#from sklearn.metrics import classification_report
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import rmsprop
import os
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [0]:
num_classes = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# The data, split between train and test sets:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


170500096/170498071 [==============================] - 287s 2us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
def create_model(batch_size=32,learn_rate=0.0001):
    model = keras.models.Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax')) 
    
    opt = keras.optimizers.rmsprop(lr=learn_rate, decay=1e-6)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
    return model


In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [0]:
batch_size = [32,64]
learn_rate = [0.0003,0.001]
param_grid = dict(learn_rate=learn_rate,batch_size=batch_size)

In [7]:
model = keras.wrappers.scikit_learn.KerasClassifier(create_model,epochs=30)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(x_train, y_train,validation_data=( x_test, y_test))




print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
  



Train on 33333 samples, validate on 10000 samples
Epoch 1/30
33333/33333 [==============================] - 18s 538us/step - loss: 1.7552 - acc: 0.3562 - val_loss: 1.4202 - val_acc: 0.4856
Epoch 2/30
33333/33333 [==============================] - 14s 427us/step - loss: 1.3940 - acc: 0.4974 - val_loss: 1.3170 - val_acc: 0.5355
Epoch 3/30
33333/33333 [==============================] - 14s 426us/step - loss: 1.2362 - acc: 0.5617 - val_loss: 1.0883 - val_acc: 0.6218
Epoch 4/30
33333/33333 [==============================] - 14s 427us/step - loss: 1.1120 - acc: 0.6077 - val_loss: 1.0003 - val_acc: 0.6494
Epoch 5/30
33333/33333 [==============================] - 14s 426us/step - loss: 1.0266 - acc: 0.6375 - val_loss: 0.9302 - val_acc: 0.6760
Epoch 6/30
33333/33333 [==============================] - 14s 426us/step - loss: 0.9584 - acc: 0.6637 - val_loss: 0.9071 - val_acc: 0.6854
Epoch 7/30
33333/33333 [==============================] - 14s 427us/step - loss: 0.9067 - acc: 0.6860 - val_loss: 0.

AttributeError: ignored